# 🚀 API GraphQL - Sistema de Gestión de Tareas

Este notebook demuestra cómo usar la **API GraphQL** del sistema de gestión de tareas. Incluye ejemplos completos de:

- ✅ Configuración e instalación
- ✅ Autenticación JWT
- ✅ Queries (consultas)
- ✅ Mutations (modificaciones)
- ✅ Cliente GraphQL Python
- ✅ Dashboard y estadísticas
- ✅ Casos de uso avanzados

## 📋 Prerrequisitos

1. **Instalar dependencias GraphQL:**
```bash
pip install strawberry-graphql[fastapi] uvicorn gql requests
```

2. **Tener el sistema core funcionando** (usuarios y tareas creados)

3. **Servidor GraphQL ejecutándose en puerto 4000**

---

## 1. 🔧 Instalar y Configurar GraphQL

Primero verificamos que tengamos todas las dependencias necesarias instaladas.

In [1]:
# Verificar e instalar dependencias
import sys
import subprocess

def install_package(package):
    """Instalar paquete si no está disponible"""
    try:
        __import__(package)
        print(f"✅ {package} ya está instalado")
    except ImportError:
        print(f"🔧 Instalando {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

# Lista de paquetes requeridos
required_packages = [
    'strawberry-graphql[fastapi]',
    'uvicorn',
    'gql[all]',
    'requests',
    'fastapi',
    'PyJWT'
]

print("🚀 Verificando dependencias para GraphQL...\n")

for package in required_packages:
    try:
        # Verificar imports principales
        if 'strawberry' in package:
            import strawberry
            print("✅ strawberry-graphql disponible")
        elif package == 'gql[all]':
            import gql
            print("✅ gql disponible")
        elif package == 'requests':
            import requests
            print("✅ requests disponible")
        elif package == 'uvicorn':
            import uvicorn
            print("✅ uvicorn disponible")
        elif package == 'fastapi':
            import fastapi
            print("✅ fastapi disponible")
        elif package == 'PyJWT':
            import jwt
            print("✅ PyJWT disponible")
    except ImportError as e:
        print(f"❌ {package}: {e}")

print("\n🔗 Agregando paths necesarios...")
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))
print("✅ Path configurado para importar módulos del proyecto")

🚀 Verificando dependencias para GraphQL...

✅ strawberry-graphql disponible
✅ uvicorn disponible
✅ gql disponible
✅ strawberry-graphql disponible
✅ uvicorn disponible
✅ gql disponible
✅ requests disponible
✅ fastapi disponible
✅ requests disponible
✅ fastapi disponible
✅ PyJWT disponible

🔗 Agregando paths necesarios...
✅ Path configurado para importar módulos del proyecto
✅ PyJWT disponible

🔗 Agregando paths necesarios...
✅ Path configurado para importar módulos del proyecto


## 2. 🏗️ Configurar Cliente GraphQL

Importamos y configuramos el cliente GraphQL que creamos.

In [2]:
# Importar cliente GraphQL
import requests
import json
from datetime import datetime

try:
    from client import TaskGraphQLClient, GraphQLError
    print("✅ Cliente GraphQL importado correctamente")
    cliente_importado = True
except ImportError as e:
    print(f"❌ Error importando cliente: {e}")
    print("🔧 Usando implementación básica...")
    cliente_importado = False
    
    # Implementación básica si no se puede importar
    class BasicGraphQLClient:
        def __init__(self, base_url="http://localhost:4000"):
            self.base_url     = base_url
            self.graphql_url  = f"{base_url}/graphql"
            self.headers      = {'Content-Type': 'application/json'}
            self.access_token = None
        
        def _execute_query(self, query, variables=None):
            payload  = {'query': query, 'variables': variables or {}}
            response = requests.post(self.graphql_url, json=payload, headers=self.headers)
            
            if response.status_code != 200:
                raise Exception(f"HTTP {response.status_code}: {response.text}")
            
            result = response.json()
            if 'errors' in result:
                raise Exception(f"GraphQL errors: {result['errors']}")
            
            return result.get('data', {})
        
        def health_check(self):
            query = "query { health }"
            result = self._execute_query(query)
            return result['health']

# Crear cliente
GRAPHQL_URL = "http://localhost:4000"

# Usar cliente apropiado según lo que esté disponible
if cliente_importado:
    client = TaskGraphQLClient(GRAPHQL_URL)
    print("🚀 Usando TaskGraphQLClient completo")
else:
    client = BasicGraphQLClient(GRAPHQL_URL)
    print("🔧 Usando BasicGraphQLClient")

print(f"🔗 Cliente GraphQL configurado para: {GRAPHQL_URL}")
print("📡 Endpoints disponibles:")
print(f"   • GraphQL API: {GRAPHQL_URL}/graphql")
print(f"   • GraphQL Playground: {GRAPHQL_URL}/graphql")
print(f"   • Documentación: {GRAPHQL_URL}/docs")

# Variables globales para el notebook
auth_token   = None
current_user = None

✅ Cliente GraphQL importado correctamente
🚀 Usando TaskGraphQLClient completo
🔗 Cliente GraphQL configurado para: http://localhost:4000
📡 Endpoints disponibles:
   • GraphQL API: http://localhost:4000/graphql
   • GraphQL Playground: http://localhost:4000/graphql
   • Documentación: http://localhost:4000/docs


## 3. 🔍 Health Check y Conexión

Verificamos que el servidor GraphQL esté funcionando correctamente.

In [3]:
# Health Check
print("🔍 Verificando conexión con el servidor GraphQL...")

try:
    health_status = client.health_check()
    print(f"✅ Servidor GraphQL: {health_status}")
    
    # Query básica de información
    query = """
        query ServerInfo {
            health
        }
    """
    
    # Ejecutar query directamente
    result = client._execute_query(query)
    print(f"📡 Respuesta del servidor: {result}")
    
except Exception as e:
    print(f"❌ Error conectando con GraphQL: {e}")
    print("🔧 Asegúrate de que el servidor esté ejecutándose:")
    print("   python graphql_api.py")
    print(f"   En: {GRAPHQL_URL}")

# Verificar endpoint REST también (si está disponible)
try:
    rest_response = requests.get(f"{GRAPHQL_URL}/health")
    if rest_response.status_code == 200:
        print(f"✅ Health endpoint REST: {rest_response.json()}")
    else:
        print(f"⚠️ Health endpoint REST: Status {rest_response.status_code}")
except:
    print("ℹ️ Endpoint REST no disponible (normal para GraphQL puro)")

print("\n🚀 ¡Conexión establecida! Puedes continuar con los ejemplos.")

🔍 Verificando conexión con el servidor GraphQL...
✅ Servidor GraphQL: GraphQL API is running! 🚀
✅ Servidor GraphQL: GraphQL API is running! 🚀
📡 Respuesta del servidor: {'health': 'GraphQL API is running! 🚀'}
📡 Respuesta del servidor: {'health': 'GraphQL API is running! 🚀'}
✅ Health endpoint REST: {'status': 'healthy', 'message': 'GraphQL API is running! 🚀', 'service': 'Sistema de Gestión de Tareas - GraphQL', 'version': '1.0.0'}

🚀 ¡Conexión establecida! Puedes continuar con los ejemplos.
✅ Health endpoint REST: {'status': 'healthy', 'message': 'GraphQL API is running! 🚀', 'service': 'Sistema de Gestión de Tareas - GraphQL', 'version': '1.0.0'}

🚀 ¡Conexión establecida! Puedes continuar con los ejemplos.


## 4. 🔐 Autenticación JWT

GraphQL requiere autenticación para la mayoría de operaciones. Vamos a hacer login y obtener tokens.

In [4]:
# Configuración de credenciales (cambiar según tu sistema)
USERNAME = "admin"  # Cambiar por un usuario existente
PASSWORD = "4321"  # Cambiar por la contraseña correcta

print("🔐 Intentando autenticación...")
print(f"👤 Usuario: {USERNAME}")

try:
    # Usar query corregida con nombres correctos y campo id
    mutation = """
        mutation Login($username: String!, $password: String!) {
            login(input: {username: $username, password: $password}) {
                accessToken
                refreshToken
                expiresIn
                usuario {
                    id
                    nombre
                    rol
                    activo
                }
            }
        }
    """
    
    variables = {'username': USERNAME, 'password': PASSWORD}
    
    # Ejecutar mutation usando el cliente base
    payload = {'query': mutation, 'variables': variables}
    response = requests.post(f"{GRAPHQL_URL}/graphql", json=payload, headers={'Content-Type': 'application/json'})
    
    if response.status_code == 200:
        result = response.json()
        
        if 'errors' in result:
            raise Exception(f"GraphQL errors: {'; '.join([e.get('message', 'Unknown') for e in result['errors']])}")
        
        # Procesar resultado exitoso
        auth_data = result['data']['login']
        
        # Guardar información de autenticación
        auth_token   = auth_data['accessToken']
        current_user = auth_data['usuario']
        
        # Configurar header para próximas requests
        client.headers['Authorization'] = f'Bearer {auth_token}'
        
        print("✅ Autenticación exitosa!")
        print(f"👤 Usuario autenticado: {current_user['nombre']}")
        print(f"🆔 ID: {current_user['id']}")
        print(f"🔑 Rol: {current_user['rol']}")
        # print(f"📧 Email: {current_user.get('email', 'No especificado')}")
        print(f"⏰ Token expira en: {auth_data['expiresIn']} segundos")
        
        # Mostrar token (primeros y últimos caracteres por seguridad)
        token_preview = f"{auth_token[:20]}...{auth_token[-10:]}"
        print(f"🎫 Token: {token_preview}")
        
    else:
        raise Exception(f"HTTP {response.status_code}: {response.text}")
    
except Exception as e:
    print(f"❌ Error de autenticación: {e}")
    print("\n🔧 Posibles soluciones:")
    print("1. Verificar que las credenciales sean correctas")
    print("2. Asegurarse de que el usuario exista en el sistema")
    print("3. Verificar que el servidor core esté ejecutándose")
    print("4. Crear un usuario admin desde la consola: python main.py")
    
    # Mensaje de ayuda para crear usuario
    print("\n💡 Para crear un usuario admin:")
    print("1. Ejecutar: python main.py")
    print("2. Opción 1: Crear usuario")
    print("3. Nombre: admin, Contraseña: admin123, Rol: administrador")

🔐 Intentando autenticación...
👤 Usuario: admin
✅ Autenticación exitosa!
👤 Usuario autenticado: admin
🆔 ID: admin
🔑 Rol: ADMIN
⏰ Token expira en: 1800 segundos
🎫 Token: eyJhbGciOiJSUzI1NiIs...fwlYCSKjcA
✅ Autenticación exitosa!
👤 Usuario autenticado: admin
🆔 ID: admin
🔑 Rol: ADMIN
⏰ Token expira en: 1800 segundos
🎫 Token: eyJhbGciOiJSUzI1NiIs...fwlYCSKjcA


## 5. 📊 Query: Dashboard Completo

Vamos a obtener todos los datos del dashboard en una sola query GraphQL. Esta es una de las principales ventajas de GraphQL: obtener múltiples recursos relacionados en una sola petición.

In [5]:
# Query del Dashboard completo
print("📊 Obteniendo datos del dashboard...")

try:
    # Query corregida para dashboard con nombres en camelCase
    dashboard_query = """
        query Dashboard {
            dashboard {
                estadisticas {
                    totalUsuarios
                    totalTareas
                    tareasPendientes
                    tareasCompletadas
                    productividadGeneral
                }
                tareasRecientes {
                    nombre
                    estado
                    fechaCreacion
                    usuariosAsignados
                }
                usuariosActivos {
                    nombre
                    rol
                    estadisticas {
                        tareasAsignadas
                        productividad
                    }
                }
                tareasUrgentes {
                    nombre
                    fechaCreacion
                    usuariosAsignados
                }
            }
        }
    """
    
    payload = {'query': dashboard_query}
    response = requests.post(f"{GRAPHQL_URL}/graphql", json=payload, headers=client.headers)
    
    if response.status_code == 200:
        result = response.json()
        
        if 'errors' in result:
            raise Exception(f"GraphQL errors: {'; '.join([e.get('message', 'Unknown') for e in result['errors']])}")
        
        dashboard_data = result['data']['dashboard']
        
        # Mostrar estadísticas generales
        stats = dashboard_data['estadisticas']
        print("📈 ESTADÍSTICAS GENERALES")
        print("=" * 40)
        print(f"👥 Total usuarios: {stats['totalUsuarios']}")
        print(f"📝 Total tareas: {stats['totalTareas']}")
        print(f"⏳ Tareas pendientes: {stats['tareasPendientes']}")
        print(f"✅ Tareas completadas: {stats['tareasCompletadas']}")
        print(f"📊 Productividad general: {stats['productividadGeneral']:.1f}%")
        
        # Mostrar tareas recientes
        print("\n🕒 TAREAS RECIENTES")
        print("=" * 40)
        tareas_recientes = dashboard_data['tareasRecientes']
        
        if tareas_recientes:
            for i, tarea in enumerate(tareas_recientes, 1):
                usuarios_asignados = tarea.get('usuariosAsignados', [])
                usuarios_str = ', '.join(usuarios_asignados) if usuarios_asignados else 'Sin asignar'
                
                print(f"{i}. {tarea['nombre']}")
                print(f"   Estado: {tarea['estado']}")
                print(f"   Creada: {tarea['fechaCreacion']}")
                print(f"   Usuarios: {usuarios_str}")
                print()
        else:
            print("No hay tareas recientes")
        
        # Mostrar usuarios activos
        print("👨‍💻 USUARIOS ACTIVOS")
        print("=" * 40)
        usuarios_activos = dashboard_data['usuariosActivos']
        
        if usuarios_activos:
            for user in usuarios_activos:
                stats_user = user.get('estadisticas', {})
                print(f"👤 {user['nombre']} ({user['rol']})")
                print(f"   Tareas asignadas: {stats_user.get('tareasAsignadas', 0)}")
                print(f"   Productividad: {stats_user.get('productividad', 0):.1f}%")
                print()
        else:
            print("No hay usuarios activos")
        
        # Mostrar tareas urgentes
        print("🚨 TAREAS URGENTES (Pendientes más antiguas)")
        print("=" * 50)
        tareas_urgentes = dashboard_data.get('tareasUrgentes', [])
        
        if tareas_urgentes:
            for i, tarea in enumerate(tareas_urgentes, 1):
                usuarios_asignados = tarea.get('usuariosAsignados', [])
                usuarios_str = ', '.join(usuarios_asignados) if usuarios_asignados else 'Sin asignar'
                
                print(f"{i}. {tarea['nombre']}")
                print(f"   Creada: {tarea['fechaCreacion']}")
                print(f"   Usuarios: {usuarios_str}")
                print()
        else:
            print("No hay tareas urgentes")
    else:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

except Exception as e:
    print(f"❌ Error obteniendo dashboard: {e}")
    if "No autorizado" in str(e):
        print("🔑 Ejecuta la celda de autenticación primero")

📊 Obteniendo datos del dashboard...
📈 ESTADÍSTICAS GENERALES
👥 Total usuarios: 33
📝 Total tareas: 10
⏳ Tareas pendientes: 8
✅ Tareas completadas: 2
📊 Productividad general: 20.0%

🕒 TAREAS RECIENTES
1. Tarea belen simple
   Estado: PENDIENTE
   Creada: 2025-11-27 23:25:03
   Usuarios: Sin asignar

2. Tarea simple creada por admin
   Estado: PENDIENTE
   Creada: 2025-11-27 23:24:01
   Usuarios: Sin asignar

3. Tarea GraphQL 23:21:49
   Estado: PENDIENTE
   Creada: 2025-11-27 23:21:53
   Usuarios: Sin asignar

4. Tarea GraphQL 23:17:56
   Estado: PENDIENTE
   Creada: 2025-11-27 23:18:00
   Usuarios: Sin asignar

5. Tarea GraphQL 23:16:28
   Estado: PENDIENTE
   Creada: 2025-11-27 23:16:32
   Usuarios: Sin asignar

👨‍💻 USUARIOS ACTIVOS
👤 admin (ADMIN)
   Tareas asignadas: 0
   Productividad: 0.0%

👤 emi (USER)
   Tareas asignadas: 0
   Productividad: 0.0%

👤 juan (USER)
   Tareas asignadas: 0
   Productividad: 0.0%

👤 maria (USER)
   Tareas asignadas: 0
   Productividad: 0.0%

👤 test_user

## 6. 👥 Query: Gestión de Usuarios

Ejemplos de consultas de usuarios con diferentes filtros y opciones.

In [6]:
# Consultas de usuarios
print("👥 CONSULTAS DE USUARIOS")
print("=" * 40)

try:
    # 1. Obtener todos los usuarios
    print("1. 📋 Todos los usuarios:")
    
    query_usuarios = """
        query GetUsuarios($filtro: FiltroUsuarios) {
            usuarios(filtro: $filtro) {
                id
                nombre
                rol
                activo
                estadisticas {
                    tareasAsignadas
                    tareasCompletadas
                    productividad
                }
            }
        }
    """
    
    variables = {'filtro': {'limite': 10}}
    payload = {'query': query_usuarios, 'variables': variables}
    response = requests.post(f"{GRAPHQL_URL}/graphql", json=payload, headers=client.headers)
    
    if response.status_code == 200:
        result = response.json()
        
        if 'errors' in result:
            raise Exception(f"GraphQL errors: {'; '.join([e.get('message', 'Unknown') for e in result['errors']])}")
        
        usuarios = result['data']['usuarios']
        
        if usuarios:
            for user in usuarios:
                print(f"   👤 {user['nombre']} (ID: {user['id']}) ({user['rol']}) - Activo: {user['activo']}")
                # Email eliminado por compatibilidad con el esquema actual
                print()
        else:
            print("   No se encontraron usuarios")
        
        # 2. Filtrar solo administradores
        print("\n2. 🔑 Solo administradores:")
        
        variables_admin = {'filtro': {'rol': 'ADMIN'}}
        payload_admin = {'query': query_usuarios, 'variables': variables_admin}
        response_admin = requests.post(f"{GRAPHQL_URL}/graphql", json=payload_admin, headers=client.headers)
        
        if response_admin.status_code == 200:
            result_admin = response_admin.json()
            if 'errors' in result_admin:
                raise Exception(f"GraphQL errors: {'; '.join([e.get('message', 'Unknown') for e in result_admin['errors']])}")
            usuarios_admin = result_admin['data']['usuarios']
            if usuarios_admin:
                for admin in usuarios_admin:
                    print(f"   👤 {admin['nombre']} (ID: {admin['id']}) ({admin['rol']}) - Activo: {admin['activo']}")
                    # Email eliminado por compatibilidad
                    print()
            else:
                print("   No se encontraron administradores")
        else:
            print(f"   Error HTTP: {response_admin.status_code}")
    else:
        print(f"   Error HTTP: {response.status_code}")
except Exception as e:
    print(f"❌ Error en consulta de usuarios: {e}")

👥 CONSULTAS DE USUARIOS
1. 📋 Todos los usuarios:
   👤 admin (ID: admin) (ADMIN) - Activo: True

   👤 emi (ID: emi) (USER) - Activo: True

   👤 juan (ID: juan) (USER) - Activo: True

   👤 maria (ID: maria) (USER) - Activo: True

   👤 test_user (ID: test_user) (USER) - Activo: True

   👤 usuario_graphql_001021 (ID: usuario_graphql_001021) (USER) - Activo: True

   👤 usuario_sin_password_001023 (ID: usuario_sin_password_001023) (USER) - Activo: True

   👤 usuario_final_001025 (ID: usuario_final_001025) (USER) - Activo: True

   👤 usuario_final_001027 (ID: usuario_final_001027) (USER) - Activo: True

   👤 usuario_sin_password_001455 (ID: usuario_sin_password_001455) (USER) - Activo: True


2. 🔑 Solo administradores:
   👤 admin (ID: admin) (ADMIN) - Activo: True

   👤 emi (ID: emi) (USER) - Activo: True

   👤 juan (ID: juan) (USER) - Activo: True

   👤 maria (ID: maria) (USER) - Activo: True

   👤 test_user (ID: test_user) (USER) - Activo: True

   👤 usuario_graphql_001021 (ID: usuario_grap

## 7. 📝 Query: Gestión de Tareas

Consultas avanzadas de tareas con filtros por estado, usuario, fechas y texto.

In [7]:
# Consultas de tareas
print("📝 CONSULTAS DE TAREAS")
print("=" * 40)

try:
    # 1. Obtener todas las tareas
    print("1. 📋 Todas las tareas:")
    
    query_tareas = """
        query GetTareas($filtro: FiltroTareas) {
            tareas(filtro: $filtro) {
                id
                nombre
                descripcion
                estado
                fechaCreacion
                usuariosAsignados
            }
        }
    """
    
    variables = {'filtro': {'limite': 10}}
    payload = {'query': query_tareas, 'variables': variables}
    response = requests.post(f"{GRAPHQL_URL}/graphql", json=payload, headers=client.headers)
    
    if response.status_code == 200:
        result = response.json()
        
        if 'errors' in result:
            raise Exception(f"GraphQL errors: {'; '.join([e.get('message', 'Unknown') for e in result['errors']])}")
        
        tareas = result['data']['tareas']
        
        if tareas:
            for tarea in tareas:
                usuarios_asignados = tarea.get('usuariosAsignados', [])
                usuarios_str = ', '.join(usuarios_asignados) if usuarios_asignados else 'Sin asignar'
                duracion_str = f" ({tarea.get('duracionDias')} días)" if tarea.get('duracionDias') else ""
                
                print(f"   📌 {tarea['nombre']} (ID: {tarea['id']})")
                print(f"      Estado: {tarea['estado']}")
                print(f"      Creada: {tarea['fechaCreacion']}")
                print(f"      Usuarios: {usuarios_str}")
                print(f"      Descripción: {tarea['descripcion'][:50]}...")
                print()
        else:
            print("   No se encontraron tareas")
        
        # 2. Filtrar solo tareas pendientes
        print("\n2. ⏳ Solo tareas pendientes:")
        
        variables_pendientes = {'filtro': {'estado': 'PENDIENTE', 'limite': 5}}
        payload_pendientes = {'query': query_tareas, 'variables': variables_pendientes}
        response_pendientes = requests.post(f"{GRAPHQL_URL}/graphql", json=payload_pendientes, headers=client.headers)
        
        if response_pendientes.status_code == 200:
            result_pendientes = response_pendientes.json()
            if 'data' in result_pendientes:
                tareas_pendientes = result_pendientes['data']['tareas']
                for tarea in tareas_pendientes:
                    print(f"   ⏳ {tarea['nombre']} (ID: {tarea['id']}) - Creada: {tarea['fechaCreacion']}")
                if not tareas_pendientes:
                    print("   No hay tareas pendientes")
        
        # 3. Filtrar solo tareas finalizadas
        print("\n3. ✅ Solo tareas finalizadas:")
        
        variables_finalizadas = {'filtro': {'estado': 'FINALIZADA', 'limite': 5}}
        payload_finalizadas = {'query': query_tareas, 'variables': variables_finalizadas}
        response_finalizadas = requests.post(f"{GRAPHQL_URL}/graphql", json=payload_finalizadas, headers=client.headers)
        
        if response_finalizadas.status_code == 200:
            result_finalizadas = response_finalizadas.json()
            if 'data' in result_finalizadas:
                tareas_finalizadas = result_finalizadas['data']['tareas']
                for tarea in tareas_finalizadas:
                    duracion = f" en {tarea.get('duracionDias')} días" if tarea.get('duracionDias') else ""
                    print(f"   ✅ {tarea['nombre']} (ID: {tarea['id']}) - Estado: {tarea['estado']}")
                if not tareas_finalizadas:
                    print("   No hay tareas finalizadas")
        
        # 4. Buscar tareas por texto
        print("\n4. 🔎 Búsqueda por texto (buscar 'api'):")
        
        variables_busqueda = {'filtro': {'textoBusqueda': 'api', 'limite': 5}}
        payload_busqueda = {'query': query_tareas, 'variables': variables_busqueda}
        response_busqueda = requests.post(f"{GRAPHQL_URL}/graphql", json=payload_busqueda, headers=client.headers)
        
        if response_busqueda.status_code == 200:
            result_busqueda = response_busqueda.json()
            if 'data' in result_busqueda:
                tareas_busqueda = result_busqueda['data']['tareas']
                if tareas_busqueda:
                    for tarea in tareas_busqueda:
                        print(f"   🎯 {tarea['nombre']} (ID: {tarea['id']}) - {tarea['estado']}")
                else:
                    print("   No se encontraron tareas con 'api'")
        
        # 5. Obtener detalle de tarea específica
        if tareas:
            primera_tarea = tareas[0]
            print(f"\n5. 🔍 Detalle de la tarea '{primera_tarea['nombre']}':")
            
            query_tarea = """
                query GetTarea($nombre: String!) {
                    tarea(nombre: $nombre) {
                        id
                        nombre
                        descripcion
                        estado
                        fechaCreacion
                        fechaFinalizacion
                        usuariosAsignados
                        comentarios {
                            texto
                            autor
                            fecha
                        }
                    }
                }
            """
            
            variables_tarea = {'nombre': primera_tarea['nombre']}
            payload_tarea = {'query': query_tarea, 'variables': variables_tarea}
            response_tarea = requests.post(f"{GRAPHQL_URL}/graphql", json=payload_tarea, headers=client.headers)
            
            if response_tarea.status_code == 200:
                result_tarea = response_tarea.json()
                if 'data' in result_tarea and result_tarea['data']['tarea']:
                    tarea_detalle = result_tarea['data']['tarea']
                    print(f"   ID: {tarea_detalle['id']}")
                    print(f"   Nombre: {tarea_detalle['nombre']}")
                    print(f"   Descripción: {tarea_detalle['descripcion']}")
                    print(f"   Estado: {tarea_detalle['estado']}")
                    print(f"   Fecha creación: {tarea_detalle['fechaCreacion']}")
                    
                    if tarea_detalle.get('fechaFinalizacion'):
                        print(f"   Fecha finalización: {tarea_detalle['fechaFinalizacion']}")
                    if tarea_detalle.get('usuariosAsignados'):
                        usuarios_asignados = tarea_detalle['usuariosAsignados']
                        print(f"   Usuarios asignados: {', '.join(usuarios_asignados) if usuarios_asignados else 'Sin asignar'}")
                    if tarea_detalle.get('comentarios'):
                        comentarios = tarea_detalle['comentarios']
                        if comentarios:
                            print(f"   Comentarios ({len(comentarios)}):")
                            for comentario in comentarios[:3]:  # Mostrar solo primeros 3
                                print(f"     💬 {comentario['autor']}: {comentario['texto'][:50]}...")
    
    else:
        raise Exception(f"HTTP {response.status_code}: {response.text}")
except Exception as e:
    print(f"❌ Error en consultas de tareas: {e}")
    if "No autorizado" in str(e) or "Unauthorized" in str(e):
        print("🔑 Asegúrate de estar autenticado")

📝 CONSULTAS DE TAREAS
1. 📋 Todas las tareas:
   📌 app de tareas (ID: app de tareas)
      Estado: FINALIZADA
      Creada: 2025-11-23 21:34:21
      Usuarios: Sin asignar
      Descripción: aplicacion de tareas...

   📌 Desarrollo API (ID: Desarrollo API)
      Estado: PENDIENTE
      Creada: 2025-11-24 00:33:55
      Usuarios: Sin asignar
      Descripción: Desarrollar API REST para el sistema...

   📌 Testing (ID: Testing)
      Estado: FINALIZADA
      Creada: 2025-11-24 00:33:55
      Usuarios: Sin asignar
      Descripción: Realizar pruebas de la aplicación...

   📌 Documentación (ID: Documentación)
      Estado: PENDIENTE
      Creada: 2025-11-24 00:33:55
      Usuarios: Sin asignar
      Descripción: Escribir documentación técnica...

   📌 tarea_test_api_2 (ID: tarea_test_api_2)
      Estado: PENDIENTE
      Creada: 2025-11-24 00:37:09
      Usuarios: Sin asignar
      Descripción: Tarea creada desde el notebook de pruebas...

   📌 Tarea GraphQL 23:16:28 (ID: Tarea GraphQL 23:16

## 8. ✏️ Mutations: Crear Usuario (Admin)

Las mutations permiten modificar datos. Crear usuarios requiere permisos de administrador.

In [8]:
# Verificar usuario actual y sus permisos
print("🔍 VERIFICANDO USUARIO ACTUAL Y PERMISOS")
print("=" * 50)

# Verificar si estamos autenticados
if 'auth_token' in locals() and 'current_user' in locals():
    print("✅ Usuario autenticado:")
    print(f"   👤 Nombre: {current_user['nombre']}")
    print(f"   🆔 ID: {current_user['id']}")
    print(f"   🔑 Rol: {current_user['rol']}")
    print(f"   📧 Email: {current_user.get('email', 'No especificado')}")
    print(f"   ✅ Activo: {current_user['activo']}")
    
    # Verificar si es administrador
    if current_user['rol'].upper() in ['ADMIN', 'ADMINISTRADOR']:
        print("   🔓 ¡Tienes permisos de administrador!")
        usuario_es_admin = True
    else:
        print("   🔒 NO tienes permisos de administrador")
        usuario_es_admin = False
        print("\n💡 SOLUCIONES:")
        print("1. Crear/usar un usuario administrador")
        print("2. Ejecutar desde la aplicación principal: python main.py")
        print("3. Cambiar las credenciales en la celda de autenticación")
        
        # Sugerencia de credenciales de admin comunes
        print("\n🔧 Credenciales de admin típicas:")
        print("   Usuario: admin, Contraseña: admin123")
        print("   Usuario: root, Contraseña: root123") 
        print("   Usuario: administrador, Contraseña: 1234")
        
else:
    print("❌ No hay usuario autenticado")
    print("🔑 Ejecuta la celda de autenticación primero")
    usuario_es_admin = False

print(f"\n📋 Estado de autenticación: {'✅ Autenticado' if 'auth_token' in locals() else '❌ No autenticado'}")
print(f"📋 Permisos de admin: {'✅ SÍ' if locals().get('usuario_es_admin', False) else '❌ NO'}")

🔍 VERIFICANDO USUARIO ACTUAL Y PERMISOS
✅ Usuario autenticado:
   👤 Nombre: admin
   🆔 ID: admin
   🔑 Rol: ADMIN
   📧 Email: No especificado
   ✅ Activo: True
   🔓 ¡Tienes permisos de administrador!

📋 Estado de autenticación: ✅ Autenticado
📋 Permisos de admin: ✅ SÍ


In [9]:
# Consulta de usuario específico
print("🔍 CONSULTA DE USUARIO ESPECÍFICO")
print("=" * 40)

try:
    usuario_nombre = "admin"  # Cambiar por un usuario existente
    query_usuario = """
        query GetUsuario($nombre: String!) {
            usuario(nombre: $nombre) {
                nombre
                rol
                activo
                tareasAsignadas {
                    nombre
                    estado
                }
                estadisticas {
                    tareasAsignadas
                    tareasCompletadas
                    productividad
                }
            }
        }
    """
    
    variables = {'nombre': usuario_nombre}
    payload = {'query': query_usuario, 'variables': variables}
    response = requests.post(f"{GRAPHQL_URL}/graphql", json=payload, headers=client.headers)
    
    if response.status_code == 200:
        result = response.json()
        
        if 'errors' in result:
            raise Exception(f"GraphQL errors: {'; '.join([e.get('message', 'Unknown') for e in result['errors']])}")
        
        usuario = result['data']['usuario']
        if usuario:
            print(f"👤 Usuario encontrado: {usuario['nombre']}")
            print(f"   Rol: {usuario['rol']}")
            print(f"   Activo: {usuario['activo']}")
            print(f"   Tareas asignadas:")
            for tarea in usuario.get('tareasAsignadas', []):
                print(f"      - {tarea['nombre']} ({tarea['estado']})")
            print(f"   Estadísticas:")
            stats = usuario.get('estadisticas', {})
            print(f"      Tareas asignadas: {stats.get('tareasAsignadas', 0)}")
            print(f"      Tareas completadas: {stats.get('tareasCompletadas', 0)}")
            print(f"      Productividad: {stats.get('productividad', 0):.1f}%")
        else:
            print("Usuario no encontrado")
    else:
        print(f"Error HTTP: {response.status_code}")
except Exception as e:
    print(f"❌ Error en consulta de usuario: {e}")

🔍 CONSULTA DE USUARIO ESPECÍFICO
👤 Usuario encontrado: admin
   Rol: ADMIN
   Activo: True
   Tareas asignadas:
   Estadísticas:
      Tareas asignadas: 0
      Tareas completadas: 0
      Productividad: 0.0%
👤 Usuario encontrado: admin
   Rol: ADMIN
   Activo: True
   Tareas asignadas:
   Estadísticas:
      Tareas asignadas: 0
      Tareas completadas: 0
      Productividad: 0.0%


In [10]:
# DEBUG: Decodificar el token JWT para ver su contenido
print("🔍 ANÁLISIS DETALLADO DEL TOKEN JWT")
print("=" * 50)

if 'auth_token' in locals():
    try:
        import base64
        import json
        
        # Decodificar el token JWT sin verificar (solo para ver el payload)
        # Los tokens JWT tienen 3 partes separadas por puntos: header.payload.signature
        token_parts = auth_token.split('.')
        
        if len(token_parts) == 3:
            # Decodificar la parte del payload (segunda parte)
            payload = token_parts[1]
            
            # Agregar padding si es necesario
            padding = len(payload) % 4
            if padding:
                payload += '=' * (4 - padding)
            
            # Decodificar base64
            decoded_payload = base64.urlsafe_b64decode(payload)
            token_data = json.loads(decoded_payload.decode('utf-8'))
            
            print("📋 CONTENIDO DEL TOKEN JWT:")
            for key, value in token_data.items():
                print(f"   {key}: {value}")
            
            print(f"\n🔑 ROL EN EL TOKEN:")
            if 'role' in token_data:
                print(f"   role (inglés): '{token_data['role']}'")
            if 'rol' in token_data:
                print(f"   rol (español): '{token_data['rol']}'")
            
            print(f"\n🔍 COMPARACIONES:")
            role_value = token_data.get('role', 'NO_ENCONTRADO')
            rol_value = token_data.get('rol', 'NO_ENCONTRADO')
            print(f"   token_data.get('role') == 'admin': {role_value == 'admin'}")
            print(f"   token_data.get('rol') == 'admin': {rol_value == 'admin'}")
            print(f"   token_data.get('role') == 'ADMIN': {role_value == 'ADMIN'}")
            print(f"   token_data.get('rol') == 'ADMIN': {rol_value == 'ADMIN'}")
            
        else:
            print("❌ Token JWT malformado (no tiene 3 partes)")
    
    except Exception as e:
        print(f"❌ Error decodificando token: {e}")
        
else:
    print("❌ No hay token disponible")

    print(f"\n🎯 CAUSA PROBABLE DEL PROBLEMA:")
    print(f"   El middleware IsAdmin está buscando 'rol' pero el token contiene 'role'")
    print(f"   O está comparando con el valor incorrecto (admin vs ADMIN)")
    print(f"\n💡 SOLUCIÓN:")
    print(f"   Necesitamos corregir la comparación en el archivo auth.py")

🔍 ANÁLISIS DETALLADO DEL TOKEN JWT
📋 CONTENIDO DEL TOKEN JWT:
   sub: admin
   exp: 1764299002
   iat: 1764297202
   iss: sistema-tareas-api
   aud: sistema-tareas-users
   jti: 5jvfXtLWj-eWkGY5b9MMJQ
   role: admin
   type: access

🔑 ROL EN EL TOKEN:
   role (inglés): 'admin'

🔍 COMPARACIONES:
   token_data.get('role') == 'admin': True
   token_data.get('rol') == 'admin': False
   token_data.get('role') == 'ADMIN': False
   token_data.get('rol') == 'ADMIN': False


In [11]:
# PRUEBA FINAL: Crear Usuario (Sistema Core Simplificado)
print("🚀 CREACIÓN DE USUARIO - SISTEMA CORE REAL")
print("=" * 60)

# Datos del nuevo usuario (solo necesita nombre)
usuario_final = f"usuario_core_{datetime.now().strftime('%H%M%S')}"

print(f"📋 CÓMO FUNCIONA EL SISTEMA CORE:")
print(f"   ✅ Solo requiere: NOMBRE del usuario")
print(f"   ✅ Rol: Siempre 'user' por defecto")
print(f"   ✅ Contraseña: Se configura en el primer login")
print(f"   ✅ Email: NO existe en el sistema core")
print(f"   ✅ Solo el admin puede crear nuevos usuarios")

print(f"\n👤 Creando usuario: {usuario_final}")

try:
    print(f"\n🔄 Enviando petición GraphQL...")
    
    # Crear usuario con el sistema simplificado (solo nombre)
    resultado = client.crear_usuario(nombre=usuario_final)
    
    if resultado['success']:
        print("🎉 ¡ÉXITO! Usuario creado correctamente")
        print(f"   📝 Mensaje: {resultado['message']}")
        
        if resultado.get('usuario'):
            usuario = resultado['usuario']
            print(f"\n   📋 Datos del usuario creado:")
            print(f"      Nombre: {usuario['nombre']}")
            print(f"      Rol: {usuario.get('rol', 'sin especificar')}")
            print(f"      Activo: {usuario.get('activo', 'sin especificar')}")
    else:
        print(f"   ❌ Error en la creación: {resultado['message']}")
        if resultado.get('code'):
            print(f"   🔢 Código: {resultado['code']}")
except Exception as e:
    print(f"❌ Error en creación de usuario: {e}")

🚀 CREACIÓN DE USUARIO - SISTEMA CORE REAL
📋 CÓMO FUNCIONA EL SISTEMA CORE:
   ✅ Solo requiere: NOMBRE del usuario
   ✅ Rol: Siempre 'user' por defecto
   ✅ Contraseña: Se configura en el primer login
   ✅ Email: NO existe en el sistema core
   ✅ Solo el admin puede crear nuevos usuarios

👤 Creando usuario: usuario_core_233341

🔄 Enviando petición GraphQL...
🎉 ¡ÉXITO! Usuario creado correctamente
   📝 Mensaje: Usuario 'usuario_core_233341' creado exitosamente. El usuario debe configurar su contraseña en el primer login.

   📋 Datos del usuario creado:
      Nombre: usuario_core_233341
      Rol: USER
      Activo: True
🎉 ¡ÉXITO! Usuario creado correctamente
   📝 Mensaje: Usuario 'usuario_core_233341' creado exitosamente. El usuario debe configurar su contraseña en el primer login.

   📋 Datos del usuario creado:
      Nombre: usuario_core_233341
      Rol: USER
      Activo: True


In [12]:
# VERIFICACIÓN FINAL - Ejecutar DESPUÉS de reiniciar el servidor
print("🎯 VERIFICACIÓN FINAL DEL FIX")
print("=" * 50)

# Datos del nuevo usuario  
usuario_final = f"usuario_final_{datetime.now().strftime('%H%M%S')}"
usuario_password = "password123"

print(f"🚀 Intentando crear usuario: {usuario_final}")

try:
    # Re-verificar autenticación primero
    if 'current_user' in locals():
        print(f"✅ Usuario autenticado: {current_user['nombre']} (rol: {current_user['rol']})")
    else:
        print("⚠️ Re-ejecuta la celda de autenticación primero")
        raise Exception("Usuario no autenticado")
    
    # Crear usuario
    resultado = client.crear_usuario(
        nombre=usuario_final,
    )
    
    if resultado['success']:
        print("\n🎉 ¡ÉXITO COMPLETO!")
        print(f"✅ Usuario creado: {resultado['message']}")
        
        usuario_creado = resultado.get('usuario')
        if usuario_creado:
            print(f"\n📋 DATOS DEL USUARIO CREADO:")
            print(f"   👤 Nombre: {usuario_creado['nombre']}")
            print(f"   🔑 Rol: {usuario_creado.get('rol', 'No especificado')}")
            print(f"   ✅ Activo: {usuario_creado.get('activo', 'No especificado')}")
        
        # Verificar que aparece en las consultas
        print(f"\n🔍 VERIFICANDO EN CONSULTA GENERAL...")
        usuarios = client.get_usuarios(limite=5)
        usuario_encontrado = False
        
        for user in usuarios:
            if user.nombre == usuario_final:
                print(f"✅ Usuario encontrado en listado general")
                usuario_encontrado = True
                break
        
        if not usuario_encontrado:
            print("⚠️ Usuario no encontrado en listado general")
    
    else:
        print(f"❌ Error: {resultado['message']}")
        if resultado.get('code'):
            print(f"   Código: {resultado['code']}")
        print(f"\n🔧 DIAGNÓSTICO:")
        print(f"   1. Verificar que reiniciaste el servidor GraphQL")
        print(f"   2. Verificar que el archivo auth.py se guardó correctamente")
        print(f"   3. Revisar logs del servidor para más detalles")

except Exception as e:
    print(f"❌ Error: {e}")
    
    if "no autorizado" in str(e).lower() or "unauthorized" in str(e).lower():
        print(f"\n🔑 SOLUCIÓN:")
        print(f"   Re-ejecuta la celda de autenticación (celda 8)")
    elif "graphql errors" in str(e).lower():
        print(f"\n🔄 SOLUCIÓN:")
        print(f"   Asegúrate de haber reiniciado el servidor GraphQL")
    else:
        print(f"\n🔧 DIAGNÓSTICO:")
        print(f"   Verificar conectividad con el servidor GraphQL")

    print(f"\n💡 Usuario de prueba: {usuario_final}")
    print(f"🔗 Servidor GraphQL: {GRAPHQL_URL}/graphql")
    print(f"📚 Documentación: {GRAPHQL_URL}/docs")

🎯 VERIFICACIÓN FINAL DEL FIX
🚀 Intentando crear usuario: usuario_final_233343
✅ Usuario autenticado: admin (rol: ADMIN)

🎉 ¡ÉXITO COMPLETO!
✅ Usuario creado: Usuario 'usuario_final_233343' creado exitosamente. El usuario debe configurar su contraseña en el primer login.

📋 DATOS DEL USUARIO CREADO:
   👤 Nombre: usuario_final_233343
   🔑 Rol: USER
   ✅ Activo: True

🔍 VERIFICANDO EN CONSULTA GENERAL...

🎉 ¡ÉXITO COMPLETO!
✅ Usuario creado: Usuario 'usuario_final_233343' creado exitosamente. El usuario debe configurar su contraseña en el primer login.

📋 DATOS DEL USUARIO CREADO:
   👤 Nombre: usuario_final_233343
   🔑 Rol: USER
   ✅ Activo: True

🔍 VERIFICANDO EN CONSULTA GENERAL...
⚠️ Usuario no encontrado en listado general
⚠️ Usuario no encontrado en listado general


## 9. ✏️ Mutations: Crear y Gestionar Tareas

Crear tareas, asignar usuarios y actualizar estados.

In [13]:
# Gestión completa de tareas
print("📝 CREAR Y GESTIONAR TAREAS")
print("=" * 40)

# Datos de la nueva tarea
nueva_tarea_nombre = f"Tarea GraphQL {datetime.now().strftime('%H:%M:%S')}"
nueva_tarea_descripcion = "Tarea creada desde GraphQL para demostración de mutations"

try:
    # 1. CREAR NUEVA TAREA
    print(f"1. ✨ Creando tarea: {nueva_tarea_nombre}")
    
    # Obtener usuarios para asignar
    usuarios = client.get_usuarios(limite=3)
    usuarios_ids = [user.nombre for user in usuarios[:2]]  # Asignar a primeros 2 usuarios
    
    resultado_crear = client.crear_tarea(
        nombre=nueva_tarea_nombre,
        descripcion=nueva_tarea_descripcion,
        usuarios_ids=usuarios_ids
    )
    
    if resultado_crear['success']:
        print("✅ Tarea creada exitosamente!")
        print(f"   Mensaje: {resultado_crear['message']}")
        
        tarea_creada = resultado_crear['tarea']
        print(f"   Nombre: {tarea_creada['nombre']}")
        print(f"   Estado: {tarea_creada['estado']}")
        print(f"   Usuarios asignados: {len(tarea_creada.get('usuariosAsignados', []))}")
        
        # Guardar nombre para operaciones posteriores
        tarea_id = tarea_creada['nombre']
        
    else:
        print(f"❌ Error creando tarea: {resultado_crear['message']}")
        tarea_id = None
    
    # 2. ASIGNAR USUARIO ADICIONAL (si hay más usuarios)
    if tarea_id and len(usuarios) > 2:
        print(f"\n2. 👤 Asignando usuario adicional a la tarea...")
        
        usuario_adicional = usuarios[2]  # Tercer usuario
        resultado_asignar = client.asignar_usuario_tarea(
            tarea_id=tarea_id,
            usuario_id=usuario_adicional.nombre
        )
        
        if resultado_asignar['success']:
            print(f"✅ Usuario {usuario_adicional.nombre} asignado correctamente")
            print(f"   Total usuarios asignados: {len(resultado_asignar['tarea'].get('usuariosAsignados', []))}")
        else:
            print(f"❌ Error asignando usuario: {resultado_asignar['message']}")
    
    # 3. ACTUALIZAR ESTADO DE TAREA
    if tarea_id:
        print(f"\n3. 📋 Actualizando estado de la tarea a FINALIZADA...")
        
        resultado_actualizar = client.actualizar_estado_tarea(
            tarea_id=tarea_id,
            estado="FINALIZADA"
        )
        
        if resultado_actualizar['success']:
            print("✅ Estado actualizado correctamente")
            tarea_actualizada = resultado_actualizar['tarea']
            
            print(f"   Estado: {tarea_actualizada['estado']}")
            if tarea_actualizada.get('fechaFinalizacion'):
                print(f"   Finalizada: {tarea_actualizada['fechaFinalizacion']}")
            if tarea_actualizada.get('duracionDias'):
                print(f"   Duración: {tarea_actualizada['duracionDias']} días")
        else:
            print(f"❌ Error actualizando estado: {resultado_actualizar['message']}")
    
    # 4. VERIFICAR TAREA FINAL
    if tarea_id:
        print(f"\n4. 🔍 Verificando tarea final...")
        tarea_final = client.get_tarea(tarea_id)
        
        if tarea_final:
            print(f"✅ Tarea verificada: {tarea_final.nombre}")
            print(f"   Estado: {tarea_final.estado}")
            print(f"   Descripción: {tarea_final.descripcion}")
            print(f"   Usuarios asignados:")
            
            for usuario_nombre in tarea_final.usuarios_asignados:
                print(f"     👤 {usuario_nombre}")
            
            if tarea_final.fecha_finalizacion:
                print(f"   Fecha finalización: {tarea_final.fecha_finalizacion}")
            if tarea_final.duracion_dias:
                print(f"   Duración: {tarea_final.duracion_dias} días")

except Exception as e:
    print(f"❌ Error en gestión de tareas: {e}")
    print(e.args)
    # Diagnóstico específico
    if "No autorizado" in str(e):
        print("🔑 Ejecuta la celda de autenticación primero")
    elif "no encontrada" in str(e):
        print("📝 La tarea no existe o fue eliminada")
    elif "Cannot query field" in str(e):
        print("❌ Error: Se está consultando un campo inexistente en el esquema GraphQL. Corrige la query/mutation para no pedir subcampos en 'usuariosAsignados'.")
    elif "must not have a selection" in str(e):
        print("❌ Error: 'usuariosAsignados' es una lista de strings, no tiene subcampos. Elimina los subcampos en la query.")
    else:
        print("🔧 Verifica que el servidor esté ejecutándose")

print(f"\n💡 TIP: Las tareas se crean con timestamp: {nueva_tarea_nombre}")

📝 CREAR Y GESTIONAR TAREAS
1. ✨ Creando tarea: Tarea GraphQL 23:33:47
❌ Error creando tarea: Error creando tarea: 'Tarea GraphQL 23:33:47'

💡 TIP: Las tareas se crean con timestamp: Tarea GraphQL 23:33:47
❌ Error creando tarea: Error creando tarea: 'Tarea GraphQL 23:33:47'

💡 TIP: Las tareas se crean con timestamp: Tarea GraphQL 23:33:47


## 10. 🔍 Queries GraphQL Personalizadas

Ejemplos de queries GraphQL avanzadas ejecutadas directamente usando el cliente base.

In [14]:
# Queries GraphQL personalizadas avanzadas
print("🔍 QUERIES GRAPHQL AVANZADAS")
print("=" * 45)

try:
    # 1. QUERY CON FRAGMENTOS
    print("1. 📋 Query con fragmentos GraphQL:")
    
    query_con_fragmentos = """
        fragment UsuarioBasico on Usuario {
            id
            nombre
            rol
            activo
        }
        
        fragment TareaBasica on Tarea {
            id
            nombre
            fechaCreacion
        }
        
        query QueryConFragmentos {
            usuarios(filtro: {limite: 3}) {
                ...UsuarioBasico
                estadisticas {
                    tareasAsignadas
                    productividad
                }
                tareasAsignadas {
                    ...TareaBasica
                }
            }
        }
    """
    
    resultado = client._execute_query(query_con_fragmentos)
    usuarios_completos = resultado['usuarios']
    
    print(f"   ✅ Obtenidos {len(usuarios_completos)} usuarios con sus tareas")
    for usuario in usuarios_completos:
        stats = usuario.get('estadisticas', {})
        tareas = usuario.get('tareasAsignadas', [])
        print(f"      👤 {usuario['nombre']} ({usuario['rol']})")
        print(f"         Productividad: {stats.get('productividad', 0):.1f}%")
        print(f"         Tareas: {len(tareas)}")
    
    # 2. QUERY CON VARIABLES Y ALIAS
    print("\n2. 🎯 Query con variables y alias:")
    
    query_con_variables = """
        query QueryConVariables($limitePendientes: Int!, $limiteFinalizada: Int!) {
            tareasPendientes: tareas(filtro: {estado: PENDIENTE, limite: $limitePendientes}) {
                id
                nombre
                fechaCreacion
                usuariosAsignados
            }
            tareasFinalizadas: tareas(filtro: {estado: FINALIZADA, limite: $limiteFinalizada}) {
                id
                nombre
                fechaCreacion
            }
        }
    """
    
    variables = {
        'limitePendientes': 3,
        'limiteFinalizada': 3
    }
    
    resultado = client._execute_query(query_con_variables, variables)
    
    pendientes = resultado['tareasPendientes']
    finalizadas = resultado['tareasFinalizadas']
    
    print(f"   ⏳ Tareas pendientes: {len(pendientes)}")
    for tarea in pendientes:
        usuarios_str = ', '.join([u['nombre'] for u in tarea.get('usuariosAsignados', [])])
        print(f"      📝 {tarea['nombre']} - Usuarios: {usuarios_str or 'Sin asignar'}")
        usuarios_asignados = tarea.get('usuariosAsignados', [])
        usuarios_str = ', '.join(usuarios_asignados) if usuarios_asignados else 'Sin asignar'
        print(f"      📝 {tarea['nombre']} - Usuarios: {usuarios_str}")
    
    print(f"   ✅ Tareas finalizadas: {len(finalizadas)}")
    print(f"      ✅ {tarea['nombre']} - Estado: {tarea['estado']}")
    
    # 3. QUERY DE ESTADÍSTICAS DETALLADAS
    print("\n3. 📊 Query de estadísticas detalladas:")
    
    query_estadisticas = """
        query EstadisticasDetalladas {
            estadisticasGenerales {
                totalUsuarios
                usuariosActivos
                totalTareas
                tareasPendientes
                tareasCompletadas
                tareasPorUsuarioPromedio
                productividadGeneral
            }
            usuarios(filtro: {limite: 5}) {
                nombre
                rol
                estadisticas {
                    tareasAsignadas
                    tareasCompletadas
                    tareasPendientes
                    productividad
                }
            }
        }
    """
    
    resultado = client._execute_query(query_estadisticas)
    
    # Estadísticas generales
    stats_generales = resultado['estadisticasGenerales']
    print("   📈 ESTADÍSTICAS GENERALES:")
    print(f"      👥 Usuarios: {stats_generales['totalUsuarios']} (activos: {stats_generales['usuariosActivos']})")
    print(f"      📝 Tareas: {stats_generales['totalTareas']} total")
    print(f"      ⏳ Pendientes: {stats_generales['tareasPendientes']}")
    print(f"      ✅ Completadas: {stats_generales['tareasCompletadas']}")
    print(f"      📊 Productividad: {stats_generales['productividadGeneral']:.1f}%")
    print(f"      📋 Promedio tareas/usuario: {stats_generales['tareasPorUsuarioPromedio']:.1f}")
    
    # Estadísticas por usuario
    usuarios_stats = resultado['usuarios']
    print(f"\n   👥 PRODUCTIVIDAD POR USUARIO:")
    for usuario in usuarios_stats:
        stats = usuario['estadisticas']
        print(f"      👤 {usuario['nombre']} ({usuario['rol']}):")
        print(f"         Asignadas: {stats['tareasAsignadas']}")
        print(f"         Completadas: {stats['tareasCompletadas']}")
        print(f"         Pendientes: {stats['tareasPendientes']}")
        print(f"         Productividad: {stats['productividad']:.1f}%")
    
    # 4. QUERY INTROSPECTIVA (SCHEMA)
    print("\n4. 🔬 Query introspectiva del schema:")
    
    query_introspectiva = """
        query IntrospectionQuery {
            __schema {
                queryType {
                    name
                    fields {
                        name
                        description
                    }
                }
                mutationType {
                    name
                    fields {
                        name
                        description
                    }
                }
            }
        }
    """
    
    try:
        resultado = client._execute_query(query_introspectiva)
        schema_info = resultado['__schema']
        
        query_fields = schema_info['queryType']['fields']
        mutation_fields = schema_info['mutationType']['fields']
        
        print(f"   📋 Queries disponibles: {len(query_fields)}")
        for field in query_fields[:5]:  # Primeras 5
            print(f"      • {field['name']}")
        
        print(f"   ✏️ Mutations disponibles: {len(mutation_fields)}")
        for field in mutation_fields[:5]:  # Primeras 5
            print(f"      • {field['name']}")
    
    except Exception as e:
        print(f"   ⚠️ Introspection no disponible: {e}")

except Exception as e:
    print(f"❌ Error en queries avanzadas: {e}")
    if "No autorizado" in str(e):
        print("🔑 Ejecuta la celda de autenticación primero")

🔍 QUERIES GRAPHQL AVANZADAS
1. 📋 Query con fragmentos GraphQL:
   ✅ Obtenidos 3 usuarios con sus tareas
      👤 admin (ADMIN)
         Productividad: 0.0%
         Tareas: 0
      👤 emi (USER)
         Productividad: 0.0%
         Tareas: 0
      👤 juan (USER)
         Productividad: 0.0%
         Tareas: 0

2. 🎯 Query con variables y alias:
   ✅ Obtenidos 3 usuarios con sus tareas
      👤 admin (ADMIN)
         Productividad: 0.0%
         Tareas: 0
      👤 emi (USER)
         Productividad: 0.0%
         Tareas: 0
      👤 juan (USER)
         Productividad: 0.0%
         Tareas: 0

2. 🎯 Query con variables y alias:
   ⏳ Tareas pendientes: 3
      📝 Desarrollo API - Usuarios: Sin asignar
      📝 Desarrollo API - Usuarios: Sin asignar
      📝 Documentación - Usuarios: Sin asignar
      📝 Documentación - Usuarios: Sin asignar
      📝 tarea_test_api_2 - Usuarios: Sin asignar
      📝 tarea_test_api_2 - Usuarios: Sin asignar
   ✅ Tareas finalizadas: 2
❌ Error en queries avanzadas: 'estado'